In [13]:
import xmlrpc.client
from datetime import date, timedelta, datetime
import pandas as pd

# Conexión con Odoo (manteniendo tus credenciales)
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
weeks = 156

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas (account.invoice) ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),    # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas (account.invoice.line) ---

# Campos de la LÍNEA de factura. Eliminamos 'number', 'user_id', 'section_id', 'partner_id' porque irán en la factura.
line_fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date"] 

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": line_fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros de líneas...")

# --- 3) Pasar a DataFrame de líneas ---
line_df = pd.DataFrame(records).fillna(0)


# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["invoice_id","create_date"])

print(f"Total de líneas descargadas: {len(line_df)}")

# ----------------------------------------------------
# --- 4) Descargar los campos adicionales de Factura (account.invoice) ---
# ----------------------------------------------------
# Añadimos los campos que quieres: number, user_id, section_id, y también partner_id y store_id
invoice_fields = ["id", "store_id", "number", "user_id", "section_id", "partner_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids], # Solo las facturas que encontramos
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# --- 5) Procesar campos de la factura ---


# El nombre del partner es el segundo elemento de la tupla (si existe)
invoice_df["client_name_inv"] = invoice_df["partner_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)


# Separar store_id
invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar user_id (Vendedor)
invoice_df["salesperson_name"] = invoice_df["user_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar section_id (Equipo de Ventas)
invoice_df["sales_team_name"] = invoice_df["section_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar partner_id (Cliente/Partner)
invoice_df["partner_id_num"] = invoice_df["partner_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
# El nombre del partner es el segundo elemento de la tupla (si existe)
invoice_df["client_name_inv"] = invoice_df["partner_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)


# Eliminar columnas originales no deseadas o ya procesadas
invoice_df = invoice_df.drop(columns=["store_id", "user_id", "section_id", "partner_id"])


# ----------------------------------------------------
# --- 6) Fusionar DataFrames ---
# ----------------------------------------------------

# Fusionamos las líneas de factura (line_df) con los datos de las facturas (invoice_df)
df = line_df.merge(
    invoice_df, 
    left_on="invoice_id_num", 
    right_on="id", 
    how="left"
)

#6 MINUTOS

Facturas encontradas: 139042
Descargados 20000 registros de líneas...
Descargados 40000 registros de líneas...
Descargados 60000 registros de líneas...
Descargados 80000 registros de líneas...
Descargados 100000 registros de líneas...
Descargados 120000 registros de líneas...
Descargados 140000 registros de líneas...
Descargados 160000 registros de líneas...
Descargados 180000 registros de líneas...
Descargados 200000 registros de líneas...
Descargados 220000 registros de líneas...
Descargados 240000 registros de líneas...
Descargados 260000 registros de líneas...
Descargados 280000 registros de líneas...
Descargados 300000 registros de líneas...
Descargados 320000 registros de líneas...
Descargados 340000 registros de líneas...
Descargados 360000 registros de líneas...
Descargados 380000 registros de líneas...
Descargados 400000 registros de líneas...
Descargados 420000 registros de líneas...
Descargados 440000 registros de líneas...
Descargados 460000 registros de líneas...
Descargad

In [14]:
import pandas as pd

# 1. Asegurar el tipo fecha
df["date_invoice"] = pd.to_datetime(df["date_invoice"])

# 2. Filtrar entre 2023-01-01 y 2025-10-31
df_filtered = df[
    (df["date_invoice"] >= "2023-01-01") &
    (df["date_invoice"] <= "2025-10-31")
]

# Crear una nueva columna con solo el nombre limpio del cliente
df_filtered["nombre_cliente"] = df_filtered["client_name_inv"].str.replace(r"\[.*?\]\s*", "", regex=True)

# 3. Agrupar por cliente y sumar ventas
clientes = (
    df_filtered.groupby("nombre_cliente")["price_subtotal"]
    .sum()
    .reset_index()
    .rename(columns={"price_subtotal": "total_ventas"})
)

# 4. Ordenar desc para Pareto
clientes = clientes.sort_values(by="total_ventas", ascending=False)

# 5. Calcular porcentaje acumulado (PARETO)
clientes["pct_acum"] = clientes["total_ventas"].cumsum() / clientes["total_ventas"].sum()

# 6. Obtener clientes que representan el 80%
top_80 = clientes[clientes["pct_acum"] <= 0.80]

top_80

/tmp/ipykernel_59105/3219498290.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["nombre_cliente"] = df_filtered["client_name_inv"].str.replace(r"\[.*?\]\s*", "", regex=True)


,nombre_cliente,total_ventas,pct_acum
8148,LUIS DANIEL QUINTERO ROPERO,2.186037e+09,0.023761
2909,DIGABE DEL CARIBE SAS,1.477736e+09,0.039824
2531,CUMMINS DE LOS ANDES S.A.,1.470602e+09,0.055809
2810,DIDIER ANDRES VARELA MONTOYA,1.444038e+09,0.071505
210,AGROFILTER BOYACA SAS,1.355675e+09,0.086240
...,...,...,...
842,AQUATICA SURTIDOR NAUTICO SAS,1.628330e+07,0.799260
10586,REPRESENTACIONES COMERCIALES NOHRA GRANADOS RA...,1.628200e+07,0.799437
7586,LA MAQUINA SAS ZOMAC,1.624730e+07,0.799614
8559,MAQUINAS BLANCAS G Y L SAS,1.619470e+07,0.799790


In [15]:
# Lista de clientes top 80%
clientes_top = clientes["nombre_cliente"].tolist()

# Filtrar el df original solo para esos nombre_cliente
df_top_clientes = df_filtered[df_filtered["nombre_cliente"].isin(clientes_top)]
df_top_clientes["año"] = df_top_clientes["date_invoice"].dt.year

historico = (
    df_top_clientes.groupby(["nombre_cliente", "año"])["price_subtotal"]
    .sum()
    .reset_index()
    .sort_values(["nombre_cliente", "año"])
)

historico_pivot = historico.pivot_table(
    index="nombre_cliente",
    columns="año",
    values="price_subtotal",
    aggfunc="sum",
    fill_value=0
)


# Agregar columna Total
historico_pivot["Total"] = (
    historico_pivot.get(2023, 0)
    + historico_pivot.get(2024, 0)
    + historico_pivot.get(2025, 0)
)

# Ordenar por ventas totales descendente
historico_pivot = historico_pivot.sort_values(by="Total", ascending=False)

historico_pivot



año,2023,2024,2025,Total
nombre_cliente,,,,
LUIS DANIEL QUINTERO ROPERO,755837800.0,754671300.0,675527500.0,2.186037e+09
DIGABE DEL CARIBE SAS,589237894.0,724350077.0,164148408.0,1.477736e+09
CUMMINS DE LOS ANDES S.A.,454628540.0,542912796.0,473060700.0,1.470602e+09
DIDIER ANDRES VARELA MONTOYA,370636300.0,433559184.0,639842400.0,1.444038e+09
AGROFILTER BOYACA SAS,361026600.0,564801008.0,429847800.0,1.355675e+09
...,...,...,...,...
DIAZ ORJUELA EDGAR ERASMO,0.0,0.0,6700.0,6.700000e+03
EDILBERTO MORENO MOYA,0.0,6700.0,0.0,6.700000e+03
PRACTILAB S.A.S,0.0,0.0,6700.0,6.700000e+03


In [16]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

wb = Workbook()

ws1 = wb.active
ws1.title = "Top80_Clientes"
for r in dataframe_to_rows(top_80, index=False, header=True):
    ws1.append(r)

ws2 = wb.create_sheet("Historico")
for r in dataframe_to_rows(historico_pivot.reset_index(), index=False, header=True):
    ws2.append(r)

wb.save("/home/donsson/proyectos/SIMON/exports/top80_historico.xlsx")
